# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

from pathlib import Path

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


D:\Anaconda3\envs\nlp-2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [30]:
w2v_folder = Path()
raw_data_folder = w2v_folder / 'data-raw'
models_folder = w2v_folder / 'models'
path_to_data_file = raw_data_folder /'reviews_data.txt.gz'

create_new_model = False

In [17]:
# data_file=str(path_to_data_file.absolute())

In [18]:
with gzip.open (path_to_data_file, 'rb') as f:
# with open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [19]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info(f"reading file {input_file}...this may take a while")
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info (f"read {i} reviews")
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess(line)


if create_new_model:
#   read the tokenized reviews into a list
#   each review item becomes a series of words
#   so this becomes a list of lists
    documents = list (read_input (data_file))
    logging.info("Done reading data files")    
else: 
    logging.info('Not reading text files as existing model will be loaded')

2019-05-11 10:45:38,356 : INFO : reading file D:\PyProjects\_references_nb\10_ml-stats-applications\nlp-00-nlp-in-practice\0.01_word2vec\data-raw\reviews_data.txt.gz...this may take a while
2019-05-11 10:45:38,358 : INFO : read 0 reviews
2019-05-11 10:45:40,463 : INFO : read 10000 reviews
2019-05-11 10:45:42,235 : INFO : read 20000 reviews
2019-05-11 10:45:44,248 : INFO : read 30000 reviews
2019-05-11 10:45:46,284 : INFO : read 40000 reviews
2019-05-11 10:45:48,345 : INFO : read 50000 reviews
2019-05-11 10:45:50,443 : INFO : read 60000 reviews
2019-05-11 10:45:52,147 : INFO : read 70000 reviews
2019-05-11 10:45:53,741 : INFO : read 80000 reviews
2019-05-11 10:45:55,768 : INFO : read 90000 reviews
2019-05-11 10:45:57,364 : INFO : read 100000 reviews
2019-05-11 10:45:58,988 : INFO : read 110000 reviews
2019-05-11 10:46:00,584 : INFO : read 120000 reviews
2019-05-11 10:46:02,364 : INFO : read 130000 reviews
2019-05-11 10:46:04,201 : INFO : read 140000 reviews
2019-05-11 10:46:05,804 : INF

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [27]:
vector_size = 152

path_to_save_model = models_folder / f'hotel-reviews-{vector_size}-01.model'
path_to_load_model = models_folder / f'hotel-reviews-{vector_size}-01.model'

str_to_save_model = str(path_to_save_model.absolute())
str_to_load_model = str(path_to_load_model.absolute())

if create_new_model:
    model = gensim.models.Word2Vec (documents, size=vector_size, window=10, min_count=2, workers=10)
    model.train(documents,total_examples=len(documents),epochs=10)
    model.save(str_to_save_model)
else:
    model = gensim.models.Word2Vec.load(str_to_load_model)

2019-05-11 10:48:12,839 : INFO : collecting all words and their counts
2019-05-11 10:48:12,840 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-11 10:48:13,057 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-05-11 10:48:13,270 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-05-11 10:48:13,531 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-05-11 10:48:13,797 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-05-11 10:48:14,114 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-05-11 10:48:14,377 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76781 word types
2019-05-11 10:48:14,580 : INFO : PROGRESS: at sentence #70000, processed 12637525 words, keeping 83194 word types
2019-05-11 10:48:14,765 : INFO : PROG

2019-05-11 10:48:49,423 : INFO : EPOCH 2 - PROGRESS: at 50.14% examples, 1554094 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:50,428 : INFO : EPOCH 2 - PROGRESS: at 55.70% examples, 1562267 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:51,418 : INFO : EPOCH 2 - PROGRESS: at 60.86% examples, 1556007 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:52,435 : INFO : EPOCH 2 - PROGRESS: at 66.33% examples, 1556324 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:53,430 : INFO : EPOCH 2 - PROGRESS: at 70.95% examples, 1545718 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:54,450 : INFO : EPOCH 2 - PROGRESS: at 76.58% examples, 1553741 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:48:55,452 : INFO : EPOCH 2 - PROGRESS: at 81.41% examples, 1550573 words/s, in_qsize 17, out_qsize 2
2019-05-11 10:48:56,456 : INFO : EPOCH 2 - PROGRESS: at 86.53% examples, 1549640 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:48:57,469 : INFO : EPOCH 2 - PROGRESS: at 92.27% examples, 1551878

2019-05-11 10:49:38,616 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-05-11 10:49:38,623 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-05-11 10:49:38,625 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-05-11 10:49:38,626 : INFO : EPOCH - 4 : training on 41519355 raw words (30350718 effective words) took 20.0s, 1519950 effective words/s
2019-05-11 10:49:39,634 : INFO : EPOCH 5 - PROGRESS: at 5.00% examples, 1520745 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:49:40,637 : INFO : EPOCH 5 - PROGRESS: at 9.85% examples, 1551546 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:49:41,638 : INFO : EPOCH 5 - PROGRESS: at 14.22% examples, 1556391 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:49:42,639 : INFO : EPOCH 5 - PROGRESS: at 18.64% examples, 1561414 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:49:43,653 : INFO : EPOCH 5 - PROGRESS: at 22.68% examples, 1534235 words/s, in_qsize 19, out_qsize 0
2019-05-11 10

2019-05-11 10:50:27,840 : INFO : EPOCH 2 - PROGRESS: at 35.72% examples, 1441900 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:50:28,848 : INFO : EPOCH 2 - PROGRESS: at 40.46% examples, 1430252 words/s, in_qsize 20, out_qsize 0
2019-05-11 10:50:29,846 : INFO : EPOCH 2 - PROGRESS: at 45.95% examples, 1436437 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:50:30,853 : INFO : EPOCH 2 - PROGRESS: at 50.85% examples, 1434342 words/s, in_qsize 17, out_qsize 2
2019-05-11 10:50:31,848 : INFO : EPOCH 2 - PROGRESS: at 55.63% examples, 1432134 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:50:32,860 : INFO : EPOCH 2 - PROGRESS: at 60.72% examples, 1435308 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:50:33,860 : INFO : EPOCH 2 - PROGRESS: at 65.98% examples, 1440090 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:50:34,865 : INFO : EPOCH 2 - PROGRESS: at 70.35% examples, 1431263 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:50:35,893 : INFO : EPOCH 2 - PROGRESS: at 75.42% examples, 1433692

2019-05-11 10:51:21,917 : INFO : EPOCH 4 - PROGRESS: at 88.82% examples, 1418733 words/s, in_qsize 17, out_qsize 2
2019-05-11 10:51:22,917 : INFO : EPOCH 4 - PROGRESS: at 93.92% examples, 1421957 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:51:23,927 : INFO : EPOCH 4 - PROGRESS: at 98.66% examples, 1419057 words/s, in_qsize 16, out_qsize 3
2019-05-11 10:51:24,175 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-05-11 10:51:24,180 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-05-11 10:51:24,182 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-05-11 10:51:24,185 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-05-11 10:51:24,191 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-05-11 10:51:24,194 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-05-11 10:51:24,196 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-05-11 10:51:2

2019-05-11 10:52:08,801 : INFO : EPOCH 7 - PROGRESS: at 4.30% examples, 1317806 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:52:09,808 : INFO : EPOCH 7 - PROGRESS: at 8.01% examples, 1237210 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:52:10,809 : INFO : EPOCH 7 - PROGRESS: at 11.84% examples, 1284659 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:52:11,825 : INFO : EPOCH 7 - PROGRESS: at 15.55% examples, 1279241 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:52:12,832 : INFO : EPOCH 7 - PROGRESS: at 19.17% examples, 1287975 words/s, in_qsize 17, out_qsize 2
2019-05-11 10:52:13,833 : INFO : EPOCH 7 - PROGRESS: at 22.98% examples, 1298022 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:52:14,842 : INFO : EPOCH 7 - PROGRESS: at 27.50% examples, 1314797 words/s, in_qsize 17, out_qsize 2
2019-05-11 10:52:15,844 : INFO : EPOCH 7 - PROGRESS: at 32.29% examples, 1313010 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:52:16,874 : INFO : EPOCH 7 - PROGRESS: at 37.23% examples, 1324081 w

2019-05-11 10:53:03,832 : INFO : EPOCH 9 - PROGRESS: at 46.52% examples, 1318729 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:53:04,846 : INFO : EPOCH 9 - PROGRESS: at 51.42% examples, 1326229 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:53:05,848 : INFO : EPOCH 9 - PROGRESS: at 55.31% examples, 1313514 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:53:06,848 : INFO : EPOCH 9 - PROGRESS: at 59.62% examples, 1308262 words/s, in_qsize 19, out_qsize 2
2019-05-11 10:53:07,859 : INFO : EPOCH 9 - PROGRESS: at 64.34% examples, 1308339 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:53:08,863 : INFO : EPOCH 9 - PROGRESS: at 68.92% examples, 1311935 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:53:09,870 : INFO : EPOCH 9 - PROGRESS: at 73.56% examples, 1315259 words/s, in_qsize 19, out_qsize 0
2019-05-11 10:53:10,864 : INFO : EPOCH 9 - PROGRESS: at 78.09% examples, 1321556 words/s, in_qsize 18, out_qsize 1
2019-05-11 10:53:11,880 : INFO : EPOCH 9 - PROGRESS: at 82.28% examples, 1318473

Save the trained model

Word2Vec has several components: the full model (just saved) but also the **KeyedVectors** accessible by `model.wv` and that is the matrix of all the trained word vectors (one set of weight for each word in vocabulary. In this case, this is 152 weight as the model was trained with `size=152`. `model.wv` is more compact than the full model 

In [28]:
word = 'dirty'
keyedvectors = model.wv[word]
print(f'Each word such as <{word}> is represented by {len(keyedvectors)} weights. Example:')
display(keyedvectors)

Each word such as <dirty> is represented by 148 weights. Example:


array([ 0.33767405,  0.6794093 , -1.6494437 ,  2.4061284 , -3.0932152 ,
       -0.97720116,  1.4268272 ,  0.7298138 , -2.1268995 , -0.5888922 ,
       -4.146032  , -0.4465947 , -0.58673054,  1.1146352 ,  4.5479803 ,
        1.5421354 , -1.0696585 , -0.83592707, -1.2704722 ,  4.144639  ,
        1.8786309 , -3.0097766 ,  2.997035  , -2.4905162 ,  0.5071115 ,
       -0.7466245 , -0.00872485, -0.8291812 , -1.1678741 ,  3.507877  ,
       -0.12339821, -1.2952466 , -0.5034353 ,  0.31752926, -4.2167687 ,
       -1.5882038 , -0.5982395 , -0.47037283,  3.591101  ,  3.8669853 ,
       -2.799466  ,  2.625027  ,  0.141962  ,  0.29805347, -2.7182837 ,
       -2.878581  , -0.5114344 ,  0.26465195, -3.2276485 , -4.5505657 ,
        1.1102492 ,  3.065824  , -2.892303  ,  0.38926795, -1.639663  ,
        3.3078532 ,  1.3884318 , -2.67059   ,  3.959185  ,  1.0809343 ,
       -1.9843053 , -0.7903039 , -0.48452562, -3.772664  ,  1.5287273 ,
       -1.0552201 , -3.2674336 ,  0.57016224,  3.9062545 ,  2.57

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [29]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

2019-05-11 10:54:12,597 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8638341426849365),
 ('unclean', 0.7788464426994324),
 ('stained', 0.7784373760223389),
 ('grubby', 0.7623147964477539),
 ('dusty', 0.760489284992218),
 ('smelly', 0.7517919540405273),
 ('dingy', 0.7328144311904907),
 ('disgusting', 0.721342146396637),
 ('mouldy', 0.7155332565307617),
 ('gross', 0.7071250081062317)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [31]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1, topn=6)

[('courteous', 0.9135578274726868),
 ('friendly', 0.8357058167457581),
 ('cordial', 0.7961171865463257),
 ('professional', 0.7880101203918457),
 ('curteous', 0.7746528387069702),
 ('attentive', 0.768876314163208)]

In [32]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('gaulle', 0.637433648109436),
 ('germany', 0.6352967023849487),
 ('canada', 0.6226556897163391),
 ('spain', 0.6202784776687622),
 ('mexico', 0.6196963787078857),
 ('england', 0.6120783090591431)]

In [33]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.8334214687347412),
 ('amazed', 0.79375159740448),
 ('astonished', 0.7687221169471741),
 ('stunned', 0.765560507774353),
 ('appalled', 0.7418582439422607),
 ('dismayed', 0.7278842926025391)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [34]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('mattress', 0.7131335735321045),
 ('duvet', 0.7086372971534729),
 ('blanket', 0.69327712059021),
 ('quilt', 0.6866929531097412),
 ('matress', 0.6786055564880371),
 ('pillowcase', 0.6777018308639526),
 ('sheets', 0.6306957006454468),
 ('quilts', 0.6306055784225464),
 ('pillows', 0.6297816038131714),
 ('foam', 0.6170405149459839)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [35]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7517919234077541

In [36]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

0.9999999999999998

In [37]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.2742473188979062

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [38]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [39]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
